### Import necessary libraries and modules

In [1]:
import os
import pickle
import time
from collections import OrderedDict
from typing import (
    List, Tuple, Dict, Optional, Callable, Union
)
import tenseal as ts

import numpy as np
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import flwr as fl
from flwr.common import (
    Metrics, EvaluateIns, EvaluateRes, FitIns, FitRes, MetricsAggregationFn, 
    Scalar, logger, ndarrays_to_parameters_custom, parameters_to_ndarrays_custom,
    Parameters, NDArrays
)
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager
from flwr.server.strategy.aggregate import weighted_loss_avg
from logging import WARNING
import pennylane as qml

from utils import *

### Creation of FHE Keys

In [2]:
def combo_keys(client_path="secret.pkl", server_path="server_key.pkl"):
    """
    To create the public/private keys combination
    args:
        client_path: path to save the secret key (str)
        server_path: path to save the server public key (str)
    """
    context_client = security.context()
    security.write_query(client_path, {"contexte": context_client.serialize(save_secret_key=True)})
    security.write_query(server_path, {"contexte": context_client.serialize()})

    _, context_client = security.read_query(client_path)
    _, context_server = security.read_query(server_path)

    context_client = ts.context_from(context_client)
    context_server = ts.context_from(context_server)
    print("Is the client context private?", ("Yes" if context_client.is_private() else "No"))
    print("Is the server context private?", ("Yes" if context_server.is_private() else "No"))


secret_path = "secret.pkl"
public_path = "server_key.pkl"
if os.path.exists(secret_path):
    print("it exists")
    _, context_client = security.read_query(secret_path)

else:
    combo_keys(client_path=secret_path, server_path=public_path)

n_qubits = 6
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}

it exists


### Creation of Model Architecture

In [3]:
dev = qml.device("default.qubit.torch", wires=n_qubits)
    
@qml.qnode(dev, interface='torch')
def quantum_net(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits)) 
    qml.BasicEntanglerLayers(weights,wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class Net(nn.Module):
    """
    A simple CNN model

    Args:
        num_classes: An integer indicating the number of classes in the dataset.
    """
    def __init__(self, num_classes=10) -> None:
        super(Net, self).__init__()
        # 3 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(3, 6, 5)
        # Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
        # 6 input image channel, 16 output channels, 5x5 square convolution
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_qubits)  # 1 if num_classes <= 2 else num_classes)
        self.qnn = qml.qnn.TorchLayer(quantum_net, weight_shapes)
        self.fc4 = nn.Linear(n_qubits, num_classes)
        self.softmax = nn.Softmax(dim=1)
        

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the neural network
        """
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten the tensor into a vector
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # output layer
        x = self.fc3(x)
        x = self.qnn(x)
        x = self.softmax(self.fc4(x))
        return x

### Define the FlowerClient class for federated learning

In [4]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, device, batch_size, save_results, matrix_path, roc_path,
                 yaml_path, he, classes, context_client):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.cid = cid
        self.device = device
        self.batch_size = batch_size
        self.save_results = save_results
        self.matrix_path = matrix_path
        self.roc_path = roc_path
        self.yaml_path = yaml_path
        self.he = he
        self.classes = classes
        self.context_client = context_client

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters2(self.net, self.context_client)

    def fit(self, parameters, config):
        server_round = config['server_round']
        local_epochs = config['local_epochs']
        lr = float(config["learning_rate"])

        print(f'[Client {self.cid}, round {server_round}] fit, config: {config}')

        set_parameters(self.net, parameters, self.context_client)

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.net.parameters(), lr=lr, momentum=0.9)

        results = engine.train(self.net, self.trainloader, self.valloader, optimizer=optimizer, loss_fn=criterion,
                               epochs=local_epochs, device=self.device)

        if self.save_results:
            save_graphs(self.save_results, local_epochs, results, f"_Client {self.cid}")

        return get_parameters2(self.net, self.context_client), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters, self.context_client)

        loss, accuracy, y_pred, y_true, y_proba = engine.test(self.net, self.valloader,
                                                              loss_fn=torch.nn.CrossEntropyLoss(), device=self.device)

        if self.save_results:
            os.makedirs(self.save_results, exist_ok=True)
            if self.matrix_path:
                save_matrix(y_true, y_pred, self.save_results + self.matrix_path, self.classes)
            if self.roc_path:
                save_roc(y_true, y_proba, self.save_results + self.roc_path, len(self.classes))

        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

### Define the client_common function to set up the Flower client

In [5]:
def client_common(cid, model_save, path_yaml, path_roc, results_save, path_matrix,
                  batch_size, trainloaders, valloaders, DEVICE, CLASSES,
                  he=False, secret_path="", server_path=""):
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    context_client = None
    net = Net(num_classes=len(CLASSES)).to(DEVICE)

    if he:
        print("Run with homomorphic encryption")
        if os.path.exists(secret_path):
            with open(secret_path, 'rb') as f:
                query = pickle.load(f)
            context_client = ts.context_from(query["contexte"])
        else:
            context_client = security.context()
            with open(secret_path, 'wb') as f:
                encode = pickle.dumps({"contexte": context_client.serialize(save_secret_key=True)})
                f.write(encode)
        secret_key = context_client.secret_key()
    else:
        print("Run WITHOUT homomorphic encryption")

    if os.path.exists(model_save):
        print(" To get the checkpoint")
        checkpoint = torch.load(model_save, map_location=DEVICE)['model_state_dict']
        if he:
            print("to decrypt model")
            server_query, server_context = security.read_query(server_path)
            server_context = ts.context_from(server_context)
            for name in checkpoint:
                print(name)
                checkpoint[name] = torch.tensor(
                    security.deserialized_layer(name, server_query[name], server_context).decrypt(secret_key)
                )
        net.load_state_dict(checkpoint)

    return FlowerClient(cid, net, trainloader, valloader, device=DEVICE, batch_size=batch_size,
                        matrix_path=path_matrix, roc_path=path_roc, save_results=results_save, yaml_path=path_yaml,
                        he=he, context_client=context_client, classes=CLASSES)

### Define utility functions for federated learning

In [6]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

def evaluate2(server_round: int, parameters: NDArrays,
              config: Dict[str, Scalar]) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    set_parameters(central, parameters)
    loss, accuracy, y_pred, y_true, y_proba = engine.test(central, testloader, loss_fn=torch.nn.CrossEntropyLoss(),
                                                          device=DEVICE)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

def get_on_fit_config_fn(epoch=2, lr=0.001, batch_size=32) -> Callable[[int], Dict[str, str]]:
    def fit_config(server_round: int) -> Dict[str, str]:
        config = {
            "learning_rate": str(lr),
            "batch_size": str(batch_size),
            "server_round": server_round,
            "local_epochs": epoch
        }
        return config
    return fit_config

def aggreg_fit_checkpoint(server_round, aggregated_parameters, central_model, path_checkpoint,
                          context_client=None, server_path=""):
    if aggregated_parameters is not None:
        print(f"Saving round {server_round} aggregated_parameters...")
        aggregated_ndarrays: List[np.ndarray] = parameters_to_ndarrays_custom(aggregated_parameters, context_client)
        if context_client:
            def serialized(key, matrix):
                if key == 'fc3.weight':
                    return matrix.serialize()
                else:
                    return matrix
            server_response = {
                **{key: serialized(key, aggregated_ndarrays[i]) for i, key in enumerate(central_model.state_dict().keys())},
                "contexte": server_context.serialize()
            }
            security.write_query(server_path, server_response)
        else:
            params_dict = zip(central_model.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            central_model.load_state_dict(state_dict, strict=True)
            if path_checkpoint:
                torch.save({
                    'model_state_dict': central_model.state_dict(),
                }, path_checkpoint)

### Define the FedCustom strategy class

In [7]:
# A Strategy from scratch with the same sampling of the clients as it is in FedAvg
# and then change the configuration dictionary
class FedCustom(fl.server.strategy.Strategy):
    def __init__(
            self,
            fraction_fit: float = 1.0,
            fraction_evaluate: float = 1.0,
            min_fit_clients: int = 2,
            min_evaluate_clients: int = 2,
            min_available_clients: int = 2,
            evaluate_fn: Optional[
                    Callable[[int, NDArrays, Dict[str, Scalar]], Optional[Tuple[float, Dict[str, Scalar]]]]
                ] = None,
            on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
            on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
            accept_failures: bool = True,
            initial_parameters: Optional[Parameters] = None,
            fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
            evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
            context_client=None
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn,
        self.accept_failures = accept_failures
        self.initial_parameters = initial_parameters
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn
        self.evaluate_metrics_aggregation_fn = evaluate_metrics_aggregation_fn
        self.context_client = context_client

    def __repr__(self) -> str:
        # Same function as FedAvg(Strategy)
        return f"FedCustom (accept_failures={self.accept_failures})"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Same function as FedAvg(Strategy)
        initial_parameters = self.initial_parameters
        self.initial_parameters = None  # Don't keep initial parameters in memory
        return initial_parameters

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        # Same function as FedAvg(Strategy)
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )

        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        # Create custom configs
        n_clients = len(clients)
        half_clients = n_clients // 2
        # Custom fit config function provided
        standard_lr = lr
        higher_lr = 0.003
        config = {"server_round": server_round, "local_epochs": 1}
        if self.on_fit_config_fn is not None:
            # Custom fit config function provided
            config = self.on_fit_config_fn(server_round)

        # fit_ins = FitIns(parameters, config)
        # Return client/config pairs
        fit_configurations = []
        for idx, client in enumerate(clients):
            config["learning_rate"] = standard_lr if idx < half_clients else higher_lr
            """
            Each pair of (ClientProxy, FitRes) constitutes 
            a successful update from one of the previously selected clients.
            """
            fit_configurations.append(
                (
                    client,
                    FitIns(
                        parameters,
                        config
                    )
                )
            )
        # Successful updates from the previously selected and configured clients
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average. (each round)"""
        # Same function as FedAvg(Strategy)
        if not results:
            return None, {}

        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Convert results parameters --> array matrix
        weights_results = [
            (parameters_to_ndarrays_custom(fit_res.parameters, self.context_client), fit_res.num_examples)
            for _, fit_res in results
        ]

        # Aggregate parameters using weighted average between the clients and convert back to parameters object (bytes)
        parameters_aggregated = ndarrays_to_parameters_custom(aggregate_custom(weights_results))

        metrics_aggregated = {}
        # Aggregate custom metrics if aggregation fn was provided
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)

        elif server_round == 1:  # Only log this warning once
            logger.log(WARNING, "No fit_metrics_aggregation_fn provided")

        # Same function as SaveModelStrategy(fl.server.strategy.FedAvg)
        """Aggregate model weights using weighted average and store checkpoint"""
        aggreg_fit_checkpoint(server_round, parameters_aggregated, central, model_save,
                              self.context_client, path_crypted)
        return parameters_aggregated, metrics_aggregated

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        # Same function as FedAvg(Strategy)
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Same function as FedAvg(Strategy)
        # Do not configure federated evaluation if fraction eval is 0.
        if self.fraction_evaluate == 0.0:
            return []

        # Parameters and config
        config = {}  # {"server_round": server_round, "local_epochs": 1}

        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )

        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        # Each pair of (ClientProxy, FitRes) constitutes a successful update from one of the previously selected clients
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""
        # Same function as FedAvg(Strategy)
        if not results:
            return None, {}

        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Aggregate loss
        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )

        metrics_aggregated = {}
        # Aggregate custom metrics if aggregation fn was provided
        if self.evaluate_metrics_aggregation_fn:
            eval_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.evaluate_metrics_aggregation_fn(eval_metrics)

        # Only log this warning once
        elif server_round == 1:
            logger.log(WARNING, "No evaluate_metrics_aggregation_fn provided")

        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""
        # Same function as FedAvg(Strategy)
        if self.evaluate_fn is None:
            # Let's assume we won't perform the global model evaluation on the server side.
            return None

        # if we have a global model evaluation on the server side :
        parameters_ndarrays = parameters_to_ndarrays_custom(parameters, self.context_client)
        eval_res = self.evaluate_fn(server_round, parameters_ndarrays, {})

        # if you haven't results
        if eval_res is None:
            return None

        loss, metrics = eval_res
        return loss, metrics

### Set up the federated learning strategy

In [8]:
# Set up your variables directly
he = False
data_path = 'data/'
dataset = 'cifar'
yaml_path = './results/FL/results.yml'
seed = 42
num_workers = 0
max_epochs = 5
batch_size = 256
length = 32
split = 10
device = 'gpu'
number_clients = 10
save_results = 'results/FL/'
matrix_path = 'confusion_matrix.png'
roc_path = 'roc.png'
model_save = 'cifar_fl.pt'
min_fit_clients = 10
min_avail_clients = 10
min_eval_clients = 10
rounds = 2
frac_fit = 1.0
frac_eval = 0.5
lr = 1e-4

In [9]:
server_context = None
DEVICE = torch.device(choice_device(device))
CLASSES = classes_string(dataset)
central = Net(num_classes=len(CLASSES)).to(DEVICE)

In [10]:
strategy = FedCustom(
    fraction_fit=frac_fit,
    fraction_evaluate=frac_eval,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_eval_clients if min_eval_clients else number_clients // 2,
    min_available_clients=min_avail_clients,
    evaluate_metrics_aggregation_fn=weighted_average,
    initial_parameters=ndarrays_to_parameters_custom(get_parameters2(central)),
    evaluate_fn=None if he else evaluate2,
    on_fit_config_fn=get_on_fit_config_fn(epoch=max_epochs, batch_size=batch_size),
    context_client=server_context
)

In [11]:
trainloaders, valloaders, testloader = data_setup.load_datasets(num_clients=number_clients,
                                                                batch_size=batch_size,
                                                                resize=True,
                                                                seed=seed,
                                                                num_workers=num_workers,
                                                                splitter=10,
                                                                dataset=dataset,  # Use the specified dataset
                                                                data_path=data_path,
                                                                data_path_val=None)  # Use the same path for validation data

def client_fn(cid: str) -> FlowerClient:
    return client_common(cid,
                         model_save, path_yaml, path_roc, results_save, path_matrix,
                         batch_size, trainloaders, valloaders, DEVICE, CLASSES, he, secret_path, server_path)

cifar
Files already downloaded and verified
Files already downloaded and verified
The training set is created for the classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### Define the client_fn function and set up the simulation

In [12]:
import warnings
warnings.simplefilter("ignore")

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)
print(f"Training on {DEVICE}")

client_resources = None

if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

model_save = model_save
path_yaml = yaml_path
path_roc = roc_path
results_save = save_results
path_matrix = matrix_path
batch_size = batch_size
he = he
secret_path = 'secret.pkl'
server_path = 'secret.pkl'
path_crypted = 'server.pkl'

print("Start simulation")
start_simulation = time.time()
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=number_clients,
    config=fl.server.ServerConfig(num_rounds=rounds),
    strategy=strategy,
    client_resources=client_resources
)
print(f"Simulation Time = {time.time() - start_simulation} seconds")

INFO flwr 2024-07-14 01:21:06,292 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)


flwr 1.5.0
numpy 1.24.3
torch 2.3.1+cu121
torchvision 0.18.1+cu121
Training on cuda:0
Start simulation


2024-07-14 01:21:08,080	INFO worker.py:1788 -- Started a local Ray instance.
INFO flwr 2024-07-14 01:21:11,997 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 6337309902.0, 'CPU': 20.0, 'accelerator_type:G': 1.0, 'node:192.168.0.156': 1.0, 'GPU': 1.0, 'object_store_memory': 3168654950.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-07-14 01:21:11,997 | server.py:89 | Initializing global parameters
INFO flwr 2024-07-14 01:21:11,998 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2024-07-14 01:21:11,998 | server.py:91 | Evaluating initial parameters


Updated model


INFO flwr 2024-07-14 01:21:13,871 | server.py:94 | initial parameters (loss, other metrics): 2.302820509672165, {'accuracy': 9.912109375}
INFO flwr 2024-07-14 01:21:13,872 | server.py:104 | FL starting
DEBUG flwr 2024-07-14 01:21:13,873 | server.py:222 | fit_round 1: strategy sampled 10 clients (out of 10)


...

(launch_and_fit pid=114875) 	Train Epoch: 5 	Train_loss: 2.3026 | Train_acc: 10.6542 % | Validation_loss: 2.3053 | Validation_acc: 6.6374 %
(launch_and_fit pid=114875) save graph in  results/FL/
Saving round 2 aggregated_parameters...
Updated model


INFO flwr 2024-07-14 01:23:46,159 | server.py:125 | fit progress: (2, 2.3028182208538057, {'accuracy': 9.912109375}, 152.2867019349942)
DEBUG flwr 2024-07-14 01:23:46,160 | server.py:173 | evaluate_round 2: strategy sampled 10 clients (out of 10)


Server-side evaluation loss 2.3028182208538057 / accuracy 9.912109375
(launch_and_evaluate pid=114875) Run WITHOUT homomorphic encryption
(launch_and_evaluate pid=114875)  To get the checkpoint
(launch_and_evaluate pid=114875) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=114875) Updated model
(launch_and_evaluate pid=114875) Run WITHOUT homomorphic encryption
(launch_and_evaluate pid=114875)  To get the checkpoint
(launch_and_evaluate pid=114875) [Client 6] evaluate, config: {}
(launch_and_evaluate pid=114875) Updated model
(launch_and_evaluate pid=114875) Run WITHOUT homomorphic encryption
(launch_and_evaluate pid=114875)  To get the checkpoint
(launch_and_evaluate pid=114875) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=114875) Updated model
(launch_and_evaluate pid=114875) Run WITHOUT homomorphic encryption
(launch_and_evaluate pid=114875)  To get the checkpoint
(launch_and_evaluate pid=114875) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=114875) 

DEBUG flwr 2024-07-14 01:23:57,530 | server.py:187 | evaluate_round 2 received 10 results and 0 failures
INFO flwr 2024-07-14 01:23:57,530 | server.py:153 | FL finished in 163.6578995029995
INFO flwr 2024-07-14 01:23:57,531 | app.py:225 | app_fit: losses_distributed [(1, 2.3034746170043947), (2, 2.3034716606140138)]
INFO flwr 2024-07-14 01:23:57,532 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-07-14 01:23:57,533 | app.py:227 | app_fit: metrics_distributed {'accuracy': [(1, 9.407338627049182), (2, 9.407338627049182)]}
INFO flwr 2024-07-14 01:23:57,533 | app.py:228 | app_fit: losses_centralized [(0, 2.302820509672165), (1, 2.302819347381592), (2, 2.3028182208538057)]
INFO flwr 2024-07-14 01:23:57,534 | app.py:229 | app_fit: metrics_centralized {'accuracy': [(0, 9.912109375), (1, 9.912109375), (2, 9.912109375)]}


Simulation Time = 171.24376440048218 seconds
